# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 10 2023 8:44AM,257114,10,0086370512,ISDIN Corporation,Released
1,Feb 10 2023 8:44AM,257111,10,0086370509,ISDIN Corporation,Released
2,Feb 10 2023 8:44AM,257111,10,0086370510,ISDIN Corporation,Released
3,Feb 10 2023 8:44AM,257111,10,0086369261,ISDIN Corporation,Released
4,Feb 10 2023 8:44AM,257111,10,0086369326,ISDIN Corporation,Released
5,Feb 10 2023 8:44AM,257111,10,0086369327,ISDIN Corporation,Released
6,Feb 10 2023 8:44AM,257111,10,0086369329,ISDIN Corporation,Released
7,Feb 10 2023 8:44AM,257111,10,0086369328,ISDIN Corporation,Released
8,Feb 10 2023 8:44AM,257111,10,0086369374,ISDIN Corporation,Released
9,Feb 10 2023 8:44AM,257111,10,0086369378,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,257107,Released,1
32,257108,Released,1
33,257111,Released,14
34,257113,Released,1
35,257114,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Released
id,
257107,1
257108,1
257111,14
257113,1
257114,1


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Released
id,
257004,1
257024,6
257027,1
257028,11
257030,3


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,Released
id,
257004,1
257024,6
257027,1
257028,11
257030,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Released
0,257004,1
1,257024,6
2,257027,1
3,257028,11
4,257030,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,id,Released
0,257004,1
1,257024,6
2,257027,1
3,257028,11
4,257030,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id          int64
Released    int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 10 2023 8:44AM,257114,10,ISDIN Corporation
1,Feb 10 2023 8:44AM,257111,10,ISDIN Corporation
15,Feb 10 2023 8:43AM,257113,19,"Graystone, LLC"
16,Feb 10 2023 8:35AM,257108,16,American International Chemical
17,Feb 10 2023 8:30AM,257087,10,Bio-PRF
24,Feb 10 2023 8:30AM,257080,10,Bio-PRF
29,Feb 10 2023 8:30AM,257094,10,SugarBear
30,Feb 10 2023 8:30AM,257089,10,ISDIN Corporation
34,Feb 10 2023 8:30AM,257098,10,ISDIN Corporation
35,Feb 10 2023 8:30AM,257095,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Released
0,Feb 10 2023 8:44AM,257114,10,ISDIN Corporation,1
1,Feb 10 2023 8:44AM,257111,10,ISDIN Corporation,14
2,Feb 10 2023 8:43AM,257113,19,"Graystone, LLC",1
3,Feb 10 2023 8:35AM,257108,16,American International Chemical,1
4,Feb 10 2023 8:30AM,257087,10,Bio-PRF,7
5,Feb 10 2023 8:30AM,257080,10,Bio-PRF,5
6,Feb 10 2023 8:30AM,257094,10,SugarBear,1
7,Feb 10 2023 8:30AM,257089,10,ISDIN Corporation,4
8,Feb 10 2023 8:30AM,257098,10,ISDIN Corporation,8
9,Feb 10 2023 8:30AM,257095,10,ISDIN Corporation,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released
0,Feb 10 2023 8:44AM,257114,10,ISDIN Corporation,1
1,Feb 10 2023 8:44AM,257111,10,ISDIN Corporation,14
2,Feb 10 2023 8:43AM,257113,19,"Graystone, LLC",1
3,Feb 10 2023 8:35AM,257108,16,American International Chemical,1
4,Feb 10 2023 8:30AM,257087,10,Bio-PRF,7
5,Feb 10 2023 8:30AM,257080,10,Bio-PRF,5
6,Feb 10 2023 8:30AM,257094,10,SugarBear,1
7,Feb 10 2023 8:30AM,257089,10,ISDIN Corporation,4
8,Feb 10 2023 8:30AM,257098,10,ISDIN Corporation,8
9,Feb 10 2023 8:30AM,257095,10,ISDIN Corporation,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Feb 10 2023 8:44AM,257114,10,ISDIN Corporation,1
1,Feb 10 2023 8:44AM,257111,10,ISDIN Corporation,14
2,Feb 10 2023 8:43AM,257113,19,"Graystone, LLC",1
3,Feb 10 2023 8:35AM,257108,16,American International Chemical,1
4,Feb 10 2023 8:30AM,257087,10,Bio-PRF,7


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet
Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released
0,Feb 10 2023 8:44AM,257114,10,ISDIN Corporation,1
1,Feb 10 2023 8:44AM,257111,10,ISDIN Corporation,14
2,Feb 10 2023 8:43AM,257113,19,"Graystone, LLC",1
3,Feb 10 2023 8:35AM,257108,16,American International Chemical,1
4,Feb 10 2023 8:30AM,257087,10,Bio-PRF,7


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Feb 10 2023 8:44AM,257114,10,ISDIN Corporation,1
1,Feb 10 2023 8:44AM,257111,10,ISDIN Corporation,14
2,Feb 10 2023 8:43AM,257113,19,"Graystone, LLC",1
3,Feb 10 2023 8:35AM,257108,16,American International Chemical,1
4,Feb 10 2023 8:30AM,257087,10,Bio-PRF,7


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released
Warehouse,,
10,4884370,110
12,514137,2
15,257004,1
16,771308,3
18,257106,1
19,2056759,28
20,257027,1
22,257102,1


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released
0,10,4884370,110
1,12,514137,2
2,15,257004,1
3,16,771308,3
4,18,257106,1
5,19,2056759,28
6,20,257027,1
7,22,257102,1


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released
0,10,110
1,12,2
2,15,1
3,16,3
4,18,1
5,19,28
6,20,1
7,22,1


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,110
1,12,Released,2
2,15,Released,1
3,16,Released,3
4,18,Released,1


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,22
Status,,,,,,,,
Released,110,2,1,3,1,28,1,1


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,22
0,Released,110,2,1,3,1,28,1,1


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,22
0,Released,110,2,1,3,1,28,1,1


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()